# Stack Overflow Survey Analysis with a focus on Venezuela

Venezuela is expiriencing one

## Some possible questions to answer

- At what age does the average developer started coding?
- What sort of roles do users have? How much do they earn?
- How optimistic about their future are developers?
- What was the top blocker for productivity?
- How often do users visit Stack Overflow?
- What's the education level and professional background of users?
- What's the gender breakdown?
- What cities are developers coming the most from?
- What's the distribution of ages amongst devs?
- What are the most popular programming languages?
- What is the opinion of developers on Blockchain Technology?
- What's the distribution of employment status?
- Distribution of company types
- What are the roles with the highest salary?
- 

In [1]:
import pandas as pd

In [8]:
df_2017 = pd.read_csv('./datasets/2017/survey_results_public.csv');
df_2018 = pd.read_csv('./datasets/2018/survey_results_public.csv');
df_2019 = pd.read_csv('./datasets/2019/survey_results_public.csv');

/Users/orlando/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
sc_2017 = pd.read_csv('./datasets/2017/survey_results_schema.csv');
sc_2018 = pd.read_csv('./datasets/2018/survey_results_schema.csv');
sc_2019 = pd.read_csv('./datasets/2019/survey_results_schema.csv');

In [10]:
len(df_2017), len(df_2018), len(df_2019)

(51392, 98855, 88883)

In [12]:
len(sc_2017), len(sc_2018), len(sc_2019)

(154, 129, 85)

In [13]:
sc_2019.head()

,Column,QuestionText
0,Respondent,Randomized respondent ID number (not in order ...
1,MainBranch,Which of the following options best describes ...
2,Hobbyist,Do you code as a hobby?
3,OpenSourcer,How often do you contribute to open source?
4,OpenSource,How do you feel about the quality of open sour...


In [14]:
sc_2018.head()

,Column,QuestionText
0,Respondent,Randomized respondent ID number (not in order ...
1,Hobby,Do you code as a hobby?
2,OpenSource,Do you contribute to open source projects?
3,Country,In which country do you currently reside?
4,Student,"Are you currently enrolled in a formal, degree..."


In [15]:
sc_2017.head()

,Column,Question
0,Respondent,Respondent ID number
1,Professional,Which of the following best describes you?
2,ProgramHobby,Do you program as a hobby or contribute to ope...
3,Country,In which country do you currently live?
4,University,"Are you currently enrolled in a formal, degree..."


In [22]:
len(sc_2017.Column.unique()), len(sc_2018.Column.unique()), len(sc_2019.Column.unique())

(154, 129, 85)

In [23]:
common_cols = set(sc_2017.Column.unique()) & set(sc_2018.Column.unique()) & set(sc_2019.Column.unique())

In [24]:
len(common_cols)

3

In [32]:
df_2018.Country.unique()

array(['Kenya', 'United Kingdom', 'United States', 'South Africa',
       'Nigeria', 'India', 'Spain', 'Croatia', 'Netherlands', 'Israel',
       'Sweden', 'Chile', 'Australia', 'Greece', 'Poland', 'Belgium',
       'Argentina', 'Germany', 'Russian Federation', 'Indonesia',
       'Ireland', 'France', 'Ukraine', 'Denmark', 'Dominican Republic',
       'China', 'Latvia', 'Algeria', 'Colombia', 'Japan', 'Finland',
       'Romania', 'Brazil', 'Bulgaria', 'Iran, Islamic Republic of...',
       'Nepal', 'Bangladesh', 'Pakistan', 'Canada', 'Jordan', 'Portugal',
       'Italy', 'New Zealand', 'Turkey', 'Czech Republic', 'Viet Nam',
       'Austria', 'Egypt', 'Malaysia', 'Mozambique', 'Thailand',
       'Slovakia', 'Lebanon', 'Mexico', 'Hong Kong (S.A.R.)', 'Norway',
       'Myanmar', 'Switzerland', 'South Korea', 'Hungary', 'Armenia',
       'Kazakhstan', 'United Arab Emirates', 'Singapore', 'Qatar',
       'Bosnia and Herzegovina', 'Bahrain', 'Sri Lanka', 'Cambodia',
       'Serbia', 'Philip

In [45]:
def clean_country(country):
    country = str(country)
    return 'Venezuela' if 'Venezuela' in country else country

In [46]:
df_2017.CleanCountry = df_2017.Country.apply(clean_country);
df_2018.CleanCountry = df_2018.Country.apply(clean_country);
df_2019.CleanCountry = df_2019.Country.apply(clean_country);

In [49]:
len(df_2017[df_2017.CleanCountry == 'Venezuela']), len(df_2018[df_2018.CleanCountry == 'Venezuela']), len(df_2019[df_2019.CleanCountry == 'Venezuela'])

(59, 123, 88)

In [25]:
common_cols

{'Country', 'Gender', 'Respondent'}